from https://pystan.readthedocs.io/en/latest/getting_started.html

In [ ]:
import stan
import asyncio
import nest_asyncio
nest_asyncio.apply()

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

In [ ]:
schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

In [ ]:
posterior = stan.build(schools_code, data=schools_data, random_seed=1)

In [ ]:
fit = posterior.sample(num_chains=4, num_samples=1000)

In [ ]:
eta = fit["eta"]  # array with shape (8, 4000)

In [ ]:
import pandas as pd

df = fit.to_frame()

In [ ]:
df